In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from io import StringIO

In [62]:
def fetch_data(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return response.text
web_url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
# fetch_data(web_url)

# Logs

In [63]:
def log_message(message):
    with open("./logs/Code_Logs.txt", "a") as f:
        f.write(f"{datetime.now()}: {message}\n")

# Extract

In [64]:
def extract():
    url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
    soup = BeautifulSoup(fetch_data(url), 'lxml')
    table = soup.find('span', string="By market capitalization").find_next('table')
    df = pd.read_html(str(table))[0]
    log_message("Data has been extracted successfully.")
    return df
org_df = extract()

C:\Users\CORECOM\AppData\Local\Temp\ipykernel_7480\991806852.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


# Transform

In [65]:
# currency_rates = pd.read_csv("input/currency_rate.csv")

# for _, row in currency_rates.iterrows():
#     currency = row['currency'].upper()
#     rate = row['rate']
#     org_df[f'Market cap ({currency})'] = org_df['Market cap (US$ billion)'] * rate
# print(org_df)

# org_df.to_csv("largest_banks_with_currency_conversions.csv", index=False)

In [66]:
def transform(df, csv_path):
    exchange_rate = pd.read_csv(csv_path, index_col=0).to_dict()["rate"]
    # print(exchange_rate)
    
    df["Market cap (PKR billion)s"] = df["Market cap (US$ billion)"] * exchange_rate["pkr"]
    df["Market cap (INR billion)s"] = df["Market cap (US$ billion)"] * exchange_rate["inr"]
    df["Market cap (GBP billion)s"] = df["Market cap (US$ billion)"] * exchange_rate["gbp"]
    df["Market cap (EUR billion)s"] = df["Market cap (US$ billion)"] * exchange_rate["eur"]
    log_message("Data has been transformed successfully.") 
    return df
# transform(org_df, './input/currency_rate.csv')

# Load

In [67]:
# def load():
#     data = pd.DataFrame(transform(org_df, './input/currency_rate.csv'))
#     data.to_csv("./output/American_Banks.csv", index=False, encoding="utf-8")
#     return f"Data has been successfully load!"

In [68]:
def load(df, output_csv_path):
    df.to_csv(output_csv_path)
    
    log_message("Data has been loaded successfully.")
    return None
    

In [69]:
if __name__ == "__main__":
    
    url = "https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks"
    csv_path = "./input/currency_rate.csv"
    output_csv_path = "./output/American_Banks.csv"
    
    df = extract()
    df = transform(df, csv_path)
    load(df, output_csv_path)
    # df = load()

    
    

C:\Users\CORECOM\AppData\Local\Temp\ipykernel_7480\991806852.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
